# Modeling12: 하이퍼파라미터 튜닝(1000번 반복 - 1시간)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 불러오기
data_origin = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")
data_origin

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [3]:
# 전처리를 위한 원본 데이터 복사
data = data_origin.copy()

In [4]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [5]:
downcast(data)

49.6% 압축됨


,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [6]:
data_OHE = pd.get_dummies(data, columns=['gender'], drop_first=True)

> Train/Test data split

In [7]:
# 단순 랜덤 샘플링
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(data, test_size = 0.2, random_state=42)


# 계층적 샘플링
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data_OHE, data_OHE['married']):
    sss_train_set = data_OHE.loc[train_index]
    sss_test_set = data_OHE.loc[test_index]

In [8]:
X_train = sss_train_set.drop("married", axis=1)
y_train = sss_train_set["married"].copy()

X_test = sss_test_set.drop("married", axis=1)
y_test = sss_test_set["married"].copy()

# Preprocessing

In [9]:
# KNN 결측치 대체
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

X_train_imputed = imputer.fit_transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)

X_test_imputed = imputer.transform(X_test)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

In [25]:
# min-max scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)

X_test_scaled = scaler.transform(X_test_imputed)

In [26]:
X_train_scaled

array([[0.2702703 , 0.16216218, 0.8888889 , ..., 0.4       , 0.2       ,
        1.        ],
       [0.1891892 , 0.4054054 , 0.7777778 , ..., 0.6       , 0.4       ,
        1.        ],
       [0.2702703 , 0.13513511, 0.5555556 , ..., 0.6       , 0.5       ,
        1.        ],
       ...,
       [0.24324328, 0.1081081 , 0.22222224, ..., 0.7       , 0.7       ,
        0.        ],
       [0.1891892 , 0.43243247, 0.        , ..., 0.6       , 0.6       ,
        1.        ],
       [0.24324328, 0.13513511, 0.7777778 , ..., 0.6       , 0.4       ,
        1.        ]], dtype=float32)

> 기본적인 데이터셋 준비 완료

# Model Training

### LighGBM

In [84]:
# 베이지안 최적화
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

param_bounds = {
    'n_estimators': (100, 3000),
    'max_depth': (3, 30),
    'num_leaves': (20, 1000),
    'min_child_samples': (10, 30),
    'learning_rate': (0.001, 0.3),
    'scale_pos_weight': (1, 2),
}

def eval_function(n_estimators, max_depth, num_leaves, min_child_samples, learning_rate, scale_pos_weight):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'num_leaves': int(num_leaves),
        'min_child_samples': int(min_child_samples),
        'learning_rate': learning_rate,
        'scale_pos_weight': scale_pos_weight,
    }
    lgbm = LGBMClassifier(**params, objective='binary', metric='accuracy', random_state=42, verbose=0)

    lgbm.fit(X_train_scaled, y_train)
    y_pred = lgbm.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [85]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=1000)

|   iter    |  target   | lambda_l2 | learni... | max_depth | min_ch... | n_esti... | num_le... | scale_... |
-------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] lambda_l2 is set=0.3745401188473625, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3745401188473625
[LightGBM] [Warning] lambda_l2 is set=0.3745401188473625, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3745401188473625
[LightGBM] [Warning] lambda_l2 is set=0.3745401188473625, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3745401188473625
| 1         | 0.8565    | 0.3745    | 0.4759    | 19.1      | 21.97     | 240.4     | 79.28     | 1.058     |
[LightGBM] [Warning] lambda_l2 is set=0.8661761457749352, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8661761457749352
[LightGBM] [Warning] lambda_l2 is set=0.8661761457749352, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8661761457749352


In [71]:
# 최적 파라미터 및 성능 확인
optimizer.max

{'target': 0.8655110579796772,
 'params': {'learning_rate': 0.23249734404308742,
  'max_depth': 22.600760172872402,
  'min_child_samples': 15.647056006826228,
  'n_estimators': 938.9350513373478,
  'num_leaves': 335.3256002286325,
  'scale_pos_weight': 1.5278900958622281}}

In [77]:
# 최적화된 하이퍼파라미터로 모델 재학습
best_params = {
    'n_estimators': round(optimizer.max['params']['n_estimators']),
    'max_depth': round(optimizer.max['params']['max_depth']),
    'num_leaves': round(optimizer.max['params']['num_leaves']),
    'min_child_samples': round(optimizer.max['params']['min_child_samples']),
    'learning_rate': optimizer.max['params']['learning_rate'],
    'scale_pos_weight': optimizer.max['params']['scale_pos_weight'],
}

best_lgbm = LGBMClassifier(**best_params, objective='binary', metric='accuracy', random_state=42, verbose=0)
best_lgbm.fit(X_train_scaled, y_train)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(learning_rate=0.0943516407059334, max_depth=17,
               metric='accuracy', min_child_samples=23, n_estimators=850,
               num_leaves=315, objective='binary', random_state=42,
               scale_pos_weight=1.7637166952716274, verbose=0)

In [88]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Train data 성능 평가
y_train_pred = best_lgbm.predict(X_train_scaled)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

# Test data 성능 평가
y_test_pred = best_lgbm.predict(X_test_scaled)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  1.0
F1 Score:  1.0
[[5479    0]
 [   0 1210]]
<<Test Set Performance>>
Accuracy:  0.8523610280932457
F1 Score:  0.5089463220675945
[[1298   72]
 [ 175  128]]
